In [ ]:
import pandas as pd
import numpy as np
import sys
import os
from sklearn.linear_model import LogisticRegression
import time
from joblib import Parallel, delayed

np.set_printoptions(threshold=sys.maxsize)

In [ ]:
ID = "sample"
L = 20

directory = os.path.dirname(os.getcwd())
df = pd.read_csv(directory + '\\datasets\\dna.csv', dtype={ID: object})

#df = df.iloc[0:10, :]

print(df)

In [ ]:
poss = []

for sample in df[ID].values:
    sample = list(str(sample))
    pos = np.array(['2']*L).astype(int)
    pos[:L] = sample
    poss.append(pos)

df_wos = df[df.columns[1:]]
poss = pd.DataFrame(poss)
poss = poss.replace(2, np.NaN)
df_wos_wp = pd.concat([poss, df_wos], axis=1)

pacient_columns = df.columns[1:]
pacient_vals = []
for pacient in pacient_columns:
    pacient_val = []
    for pos in range(L):
        pacient_val.append((df_wos_wp[pos]*df_wos_wp[pacient]).sum()/(df_wos_wp[pacient][~df_wos_wp[pos].isna()].sum()))
    pacient_vals.append(pacient_val)
pacient_vals = np.array(pacient_vals)

val_log = np.log(pacient_vals)

pacient_vals


In [ ]:
def count_pair(data, i, j):
    patterns = ["11", "01", "10", "00"]
    data_p = data
    data_p = data[~data[i].isna()]
    data_p = data_p[~data_p[j].isna()]
    i_col = data_p[i].astype(int).astype(str)
    j_col = data_p[j].astype(int).astype(str)
    pair_col = i_col+j_col
    res = []
    for p in patterns:
        res.append((pair_col == p).astype(int))
    return np.array(res)


pairs = []
for i in range(L):
    for j in range(L):
        if (j > i):
            pairs.append([i, j])

pacient_pair_vals = []
col_names = []
for pair in pairs:
    counts = count_pair(df_wos_wp, pair[0], pair[1])
    pacient_pair_val = []
    for pacient in pacient_columns:
        sum = df_wos_wp[pacient].sum()
        k = 0
        val = []
        for p in counts:
            val.append((p*df_wos_wp[pacient]).sum()/sum)
            col_names.append(
                f'{pacient}_{pair[0]}_{pair[1]}_{["11", "01", "10", "00"][k]}')
            k += 1
        pacient_pair_val.append(val)
    pacient_pair_vals.append(pacient_pair_val)
    
pacient_pair_vals = np.array(pacient_pair_vals)

pacient_pair_vals = pacient_pair_vals.transpose(1, 0, 2).reshape(pacient_pair_vals.shape[1],
                                                               pacient_pair_vals.shape[0] *
                                                               pacient_pair_vals.shape[2],
                                                               )

pair_val_log = np.log(pacient_pair_vals)

pacient_pair_vals


In [ ]:
labels = np.array(['p_cncr' in i for i in pacient_columns]).astype(int)

lfdf = pd.DataFrame(pair_val_log.T, columns=labels)
lfdf.replace([np.inf, -np.inf], np.nan, inplace=True)
lfdf.dropna(inplace=True)
pair_val_log = lfdf.to_numpy().T

all_log = np.concatenate([val_log, pair_val_log], axis=-1)

all_log


In [ ]:
def cros_val(feaches, labels):
    ind = np.arange(len(labels))
    result = []
    for val_ind in range(len(labels)):
        X_train = feaches[ind != val_ind]
        Y_train = labels[ind != val_ind]

        X_test = feaches[ind == val_ind]
        Y_test = labels[ind == val_ind]

        clf = LogisticRegression()
        clf.fit(X_train, Y_train)

        pred = clf.predict(X_test)
        result.append((pred == Y_test).astype(int))

    return result


result = cros_val(all_log, labels)


In [ ]:
np.mean(result)


In [ ]:
pair_list = []
for i in range(len(pair_val_log[0])):
    for j in range(len(pair_val_log[0])):
        if (i != j):
            pair_list.append(
                (np.stack([pair_val_log[:, i], pair_val_log[:, j]], axis=-1), labels))


In [ ]:
def do_job(processes, data=pair_list[:200]):
    start_time = time.time()
    res = [Parallel(n_jobs=processes)(delayed(cros_val)(s[0], s[1]) for s in data)]
    execution_time = time.time() - start_time
    return np.mean(res), execution_time


for i in range(1, 10):
    print(i, do_job(i))
